# LLM Optimization Modelling Experiment

In [661]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [762]:
problem = '''We are looking at an alkylation process which will include the following 10 variables: olefin feed (barrels per day), isobutane recycle (barrels per day), acid addition rate (thousands of pounds per day), alkylate yield (barrels per day), isobutane makeup (barrels per day), acid strength (weight per cent), motor octane number, external isobutane-to-olefin ratio, acid dilution factor and F-4 performance number. 

We want to maximize the daily profit of this alkylation process. 
The profit is defined as the revenue generated from the alkylate yield multiplied with the motor octane number, minus the operational costs, which include olefin feed, isobutane recycle, acid addition rate, and isobutane makeup. 

Relationships in terms of other variables for alkylate yield, motor octane number, acid dilution factor, and F-4 performance number can be formulated as regression formulas. 
This regression estimate can deviate in both directions from true value of these variables by 2, 1, 5 and 10 percent, respectively.
Alkylate yield is a function of olefin feed and external isobutane-to-olefine yield. Alkalyte yield equals the amount of olefin feed multiplied by the sum of 1.12, 0.13167 times the external isobutane-to-olefin ratio and -0.00667 times the external isobutane-to-olefin ratio squared.
The motor octane number is derived from the external isobutane-to-olefin ratio and the acid strength. The motor octane number is calculated as the sum of 86.35, 1.098 time external isobutane-to-olefin ratio, -0.038 times the external isobutane-to-olefin ratio squared and 0.325 times acid strength reduced by 89.
The acid dilution factor is calculated based on the F-4 performance number. The acid dillution factor is expressed as 35.82 minus 0.222 times F-4 performance number.
Lastly, the F-4 performance number depends on the motor octane number. F-4 performance number is calculated as -133 plus three times the motor octane number.

There are some additional constraints imposed by the nature of the chemical process.
Each variable has a lower and an upper bound.
The external isobutane-to-olefin ratio needs to equal the ratio of isobutane recycle plus isobutane makeup to olefin feed. 
The acid strength needs to equal the ratio of 93000 times acid addition rate to acid addition rate multiplied by acid dilution factor in addition to 1000 times acid addition rate. 
Lastly, 1.22 alkylate yield needs to be equal to the combined olefin feed and isobutane makeup.'''

## 2. Generate the mathematical model

In [763]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [764]:
#Show the resopnse in a formatted way
Markdown(response.text)

**Indices:**

*  None

**Parameters:**

*  Revenue per barrel of alkylate yield:  *r* (dollars/barrel)

**Variables:**

*  Olefin feed (barrels per day):  *OF*
*  Isobutane recycle (barrels per day): *IR*
*  Acid addition rate (thousands of pounds per day): *AAR*
*  Alkylate yield (barrels per day): *AY*
*  Isobutane makeup (barrels per day): *IM*
*  Acid strength (weight percent): *AS*
*  Motor octane number: *MON*
*  External isobutane-to-olefin ratio: *IOR*
*  Acid dilution factor: *ADF*
*  F-4 performance number: *F4*

**Objective Function:**

Maximize daily profit: 
```
Maximize  r * AY * MON - (c_OF * OF + c_IR * IR + c_AAR * AAR + c_IM * IM)
```
where:
* c_OF: Cost per barrel of olefin feed 
* c_IR: Cost per barrel of isobutane recycle
* c_AAR: Cost per thousand pounds of acid addition rate
* c_IM: Cost per barrel of isobutane makeup

**Constraints:**

1. **Alkylate yield:**
   ```
   AY = (1 + 0.13167 * IOR - 0.00667 * IOR^2) * OF
   (1 - 0.02) * AY <= (1 + 0.13167 * IOR - 0.00667 * IOR^2) * OF  <= (1 + 0.02) * AY 
   ```

2. **Motor octane number:**
   ```
   MON = 86.35 + 1.098 * IOR - 0.038 * IOR^2 + 0.325 * AS - 89
   (1 - 0.01) * MON <= 86.35 + 1.098 * IOR - 0.038 * IOR^2 + 0.325 * AS - 89 <= (1 + 0.01) * MON 
   ```

3. **Acid dilution factor:**
   ```
   ADF = 35.82 - 0.222 * F4
   (1 - 0.05) * ADF <= 35.82 - 0.222 * F4 <= (1 + 0.05) * ADF 
   ```

4. **F-4 performance number:**
   ```
   F4 = -133 + 3 * MON
   (1 - 0.1) * F4 <= -133 + 3 * MON <= (1 + 0.1) * F4
   ```

5. **External isobutane-to-olefin ratio:**
   ```
   IOR = (IR + IM) / OF
   ```

6. **Acid strength:**
   ```
   AS = (93000 * AAR) / (AAR * ADF + 1000 * AAR)
   ```

7. **Alkylate yield and feed:**
   ```
   1.22 * AY = OF + IM
   ```

8. **Variable bounds:**
   ```
   OF_lower <= OF <= OF_upper
   IR_lower <= IR <= IR_upper
   AAR_lower <= AAR <= AAR_upper
   AY_lower <= AY <= AY_upper
   IM_lower <= IM <= IM_upper
   AS_lower <= AS <= AS_upper
   MON_lower <= MON <= MON_upper
   IOR_lower <= IOR <= IOR_upper
   ADF_lower <= ADF <= ADF_upper
   F4_lower <= F4 <= F4_upper
   ```

You will need to determine the values for the lower and upper bounds of each variable (e.g., OF_lower, OF_upper) based on the specific characteristics of the alkylation process. Additionally, you will need to determine the revenue per barrel of alkylate yield, *r*, and the cost coefficients (c_OF, c_IR, c_AAR, c_IM). 

This model can be solved using optimization software to determine the values of the decision variables that maximize daily profit while satisfying the constraints of the alkylation process. 


## 3. Generate the pyomo code

In [765]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [766]:
### Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample Data - Replace with actual values
r = 100  # Revenue per barrel of alkylate yield
c_OF = 50  # Cost per barrel of olefin feed
c_IR = 45  # Cost per barrel of isobutane recycle
c_AAR = 10  # Cost per thousand pounds of acid addition rate
c_IM = 40  # Cost per barrel of isobutane makeup

OF_lower = 1000  # Lower bound of olefin feed
OF_upper = 5000  # Upper bound of olefin feed
IR_lower = 500   # Lower bound of isobutane recycle
IR_upper = 3000  # Upper bound of isobutane recycle
AAR_lower = 10    # Lower bound of acid addition rate
AAR_upper = 100   # Upper bound of acid addition rate
AY_lower = 1200  # Lower bound of alkylate yield
AY_upper = 6000  # Upper bound of alkylate yield
IM_lower = 100   # Lower bound of isobutane makeup
IM_upper = 1000  # Upper bound of isobutane makeup
AS_lower = 85    # Lower bound of acid strength
AS_upper = 95    # Upper bound of acid strength
MON_lower = 88    # Lower bound of motor octane number
MON_upper = 94    # Upper bound of motor octane number
IOR_lower = 1     # Lower bound of external isobutane-to-olefin ratio
IOR_upper = 10    # Upper bound of external isobutane-to-olefin ratio
ADF_lower = 30    # Lower bound of acid dilution factor
ADF_upper = 40    # Upper bound of acid dilution factor
F4_lower = -150  # Lower bound of F-4 performance number
F4_upper = -100   # Upper bound of F-4 performance number

# Model Creation
model = pyo.ConcreteModel()

# Variables
model.OF = pyo.Var(bounds=(OF_lower, OF_upper))
model.IR = pyo.Var(bounds=(IR_lower, IR_upper))
model.AAR = pyo.Var(bounds=(AAR_lower, AAR_upper))
model.AY = pyo.Var(bounds=(AY_lower, AY_upper))
model.IM = pyo.Var(bounds=(IM_lower, IM_upper))
model.AS = pyo.Var(bounds=(AS_lower, AS_upper))
model.MON = pyo.Var(bounds=(MON_lower, MON_upper))
model.IOR = pyo.Var(bounds=(IOR_lower, IOR_upper))
model.ADF = pyo.Var(bounds=(ADF_lower, ADF_upper))
model.F4 = pyo.Var(bounds=(F4_lower, F4_upper))

# Objective Function
model.profit = pyo.Objective(
    expr=r * model.AY * model.MON - (c_OF * model.OF + c_IR * model.IR + c_AAR * model.AAR + c_IM * model.IM),
    sense=pyo.maximize
)

# Constraints
model.alkylate_yield = pyo.Constraint(
    expr=(1 - 0.02) * model.AY <= (1 + 0.13167 * model.IOR - 0.00667 * model.IOR**2) * model.OF,
    expr=(1 + 0.13167 * model.IOR - 0.00667 * model.IOR**2) * model.OF <= (1 + 0.02) * model.AY
)

model.motor_octane_number = pyo.Constraint(
    expr=(1 - 0.01) * model.MON <= 86.35 + 1.098 * model.IOR - 0.038 * model.IOR**2 + 0.325 * model.AS - 89,
    expr=86.35 + 1.098 * model.IOR - 0.038 * model.IOR**2 + 0.325 * model.AS - 89 <= (1 + 0.01) * model.MON
)

model.acid_dilution_factor = pyo.Constraint(
    expr=(1 - 0.05) * model.ADF <= 35.82 - 0.222 * model.F4,
    expr=35.82 - 0.222 * model.F4 <= (1 + 0.05) * model.ADF
)

model.f4_performance_number = pyo.Constraint(
    expr=(1 - 0.1) * model.F4 <= -133 + 3 * model.MON,
    expr=-133 + 3 * model.MON <= (1 + 0.1) * model.F4
)

model.external_isobutane_to_olefin_ratio = pyo.Constraint(
    expr=model.IOR == (model.IR + model.IM) / model.OF
)

model.acid_strength = pyo.Constraint(
    expr=model.AS == (93000 * model.AAR) / (model.AAR * model.ADF + 1000 * model.AAR)
)

model.alkylate_yield_and_feed = pyo.Constraint(
    expr=1.22 * model.AY == model.OF + model.IM
)

# Solve the model
solver = pyo.SolverFactory('ipopt')
results = solver.solve(model)

# Print results
print(results)
print("Profit = ", model.profit())
print("OF = ", model.OF())
print("IR = ", model.IR())
print("AAR = ", model.AAR())
print("AY = ", model.AY())
print("IM = ", model.IM())
print("AS = ", model.AS())
print("MON = ", model.MON())
print("IOR = ", model.IOR())
print("ADF = ", model.ADF())
print("F4 = ", model.F4())
```

This code defines the Pyomo model for the alkylation process optimization problem. You still need to replace the sample data for the parameters and bounds with the actual values for your specific scenario. After replacing the sample data, you can run this code using a Pyomo solver to find the optimal values for the decision variables that maximize the daily profit while satisfying the constraints. 


## 4. Input problem data and try running the generated code

In [770]:
import pyomo.environ as pyo

# Sample Data - Replace with actual values
r = 100  # Revenue per barrel of alkylate yield
c_OF = 50  # Cost per barrel of olefin feed
c_IR = 45  # Cost per barrel of isobutane recycle
c_AAR = 10  # Cost per thousand pounds of acid addition rate
c_IM = 40  # Cost per barrel of isobutane makeup

OF_lower = 1000  # Lower bound of olefin feed
OF_upper = 5000  # Upper bound of olefin feed
IR_lower = 500   # Lower bound of isobutane recycle
IR_upper = 3000  # Upper bound of isobutane recycle
AAR_lower = 10    # Lower bound of acid addition rate
AAR_upper = 100   # Upper bound of acid addition rate
AY_lower = 1200  # Lower bound of alkylate yield
AY_upper = 6000  # Upper bound of alkylate yield
IM_lower = 100   # Lower bound of isobutane makeup
IM_upper = 1000  # Upper bound of isobutane makeup
AS_lower = 85    # Lower bound of acid strength
AS_upper = 95    # Upper bound of acid strength
MON_lower = 88    # Lower bound of motor octane number
MON_upper = 94    # Upper bound of motor octane number
IOR_lower = 1     # Lower bound of external isobutane-to-olefin ratio
IOR_upper = 10    # Upper bound of external isobutane-to-olefin ratio
ADF_lower = 30    # Lower bound of acid dilution factor
ADF_upper = 40    # Upper bound of acid dilution factor
F4_lower = -150  # Lower bound of F-4 performance number
F4_upper = -100   # Upper bound of F-4 performance number

# Model Creation
model = pyo.ConcreteModel()

# Variables
model.OF = pyo.Var(bounds=(OF_lower, OF_upper))
model.IR = pyo.Var(bounds=(IR_lower, IR_upper))
model.AAR = pyo.Var(bounds=(AAR_lower, AAR_upper))
model.AY = pyo.Var(bounds=(AY_lower, AY_upper))
model.IM = pyo.Var(bounds=(IM_lower, IM_upper))
model.AS = pyo.Var(bounds=(AS_lower, AS_upper))
model.MON = pyo.Var(bounds=(MON_lower, MON_upper))
model.IOR = pyo.Var(bounds=(IOR_lower, IOR_upper))
model.ADF = pyo.Var(bounds=(ADF_lower, ADF_upper))
model.F4 = pyo.Var(bounds=(F4_lower, F4_upper))

# Objective Function
model.profit = pyo.Objective(
    expr=r * model.AY * model.MON - (c_OF * model.OF + c_IR * model.IR + c_AAR * model.AAR + c_IM * model.IM),
    sense=pyo.maximize
)

# Constraints
model.alkylate_yield = pyo.Constraint(
    expr=(1 - 0.02) * model.AY <= (1 + 0.13167 * model.IOR - 0.00667 * model.IOR**2) * model.OF,
    expr=(1 + 0.13167 * model.IOR - 0.00667 * model.IOR**2) * model.OF <= (1 + 0.02) * model.AY
)

model.motor_octane_number = pyo.Constraint(
    expr=(1 - 0.01) * model.MON <= 86.35 + 1.098 * model.IOR - 0.038 * model.IOR**2 + 0.325 * model.AS - 89,
    expr=86.35 + 1.098 * model.IOR - 0.038 * model.IOR**2 + 0.325 * model.AS - 89 <= (1 + 0.01) * model.MON
)

model.acid_dilution_factor = pyo.Constraint(
    expr=(1 - 0.05) * model.ADF <= 35.82 - 0.222 * model.F4,
    expr=35.82 - 0.222 * model.F4 <= (1 + 0.05) * model.ADF
)

model.f4_performance_number = pyo.Constraint(
    expr=(1 - 0.1) * model.F4 <= -133 + 3 * model.MON,
    expr=-133 + 3 * model.MON <= (1 + 0.1) * model.F4
)

model.external_isobutane_to_olefin_ratio = pyo.Constraint(
    expr=model.IOR == (model.IR + model.IM) / model.OF
)

model.acid_strength = pyo.Constraint(
    expr=model.AS == (93000 * model.AAR) / (model.AAR * model.ADF + 1000 * model.AAR)
)

model.alkylate_yield_and_feed = pyo.Constraint(
    expr=1.22 * model.AY == model.OF + model.IM
)

# Solve the model
solver = pyo.SolverFactory('ipopt')
results = solver.solve(model)

# Print results
print(results)
print("Profit = ", model.profit())
print("OF = ", model.OF())
print("IR = ", model.IR())
print("AAR = ", model.AAR())
print("AY = ", model.AY())
print("IM = ", model.IM())
print("AS = ", model.AS())
print("MON = ", model.MON())
print("IOR = ", model.IOR())
print("ADF = ", model.ADF())
print("F4 = ", model.F4())

SyntaxError: keyword argument repeated: expr (2977587326.py, line 55)

## 5. Correct the code to verify model viability (optional)

In [773]:
import pyomo.environ as pyo
data = {
    'LB': {'x1': 0, 'x2': 0, 'x3': 0, 'x4': 0, 'x5': 0, 'x6': 85, 'x7': 90, 'x8': 3, 'x9': 1.2, 'x10': 145},
    'UB': {'x1': 2000, 'x2': 16000, 'x3': 120, 'x4': 5000, 'x5': 2000, 'x6': 93, 'x7': 90, 'x8': 12, 'x9': 4, 'x10': 162},
    'Cost': {'x1': 5.04, 'x2': 0.035, 'x3': 10.0, 'x4': 0, 'x5': 3.36}, #x4 is 0 cause it is not included in costs
    'Revenue': 0.63 
}
# Sample Data - Replace with actual values
r = 0.63  # Revenue per barrel of alkylate yield
c_OF = 5.04  # Cost per barrel of olefin feed
c_IR = 0.035  # Cost per barrel of isobutane recycle
c_AAR = 10.0  # Cost per thousand pounds of acid addition rate
c_IM = 3.36  # Cost per barrel of isobutane makeup

OF_lower = 0  # Lower bound of olefin feed
OF_upper = 2000  # Upper bound of olefin feed
IR_lower = 0   # Lower bound of isobutane recycle
IR_upper = 16000  # Upper bound of isobutane recycle
AAR_lower = 0    # Lower bound of acid addition rate
AAR_upper = 120   # Upper bound of acid addition rate
AY_lower = 0  # Lower bound of alkylate yield
AY_upper = 5000  # Upper bound of alkylate yield
IM_lower = 0   # Lower bound of isobutane makeup
IM_upper = 2000  # Upper bound of isobutane makeup
AS_lower = 85    # Lower bound of acid strength
AS_upper = 93    # Upper bound of acid strength
MON_lower = 90    # Lower bound of motor octane number
MON_upper = 90    # Upper bound of motor octane number
IOR_lower = 3     # Lower bound of external isobutane-to-olefin ratio
IOR_upper = 12   # Upper bound of external isobutane-to-olefin ratio
ADF_lower = 1.2    # Lower bound of acid dilution factor
ADF_upper = 4    # Upper bound of acid dilution factor
F4_lower = 145 # Lower bound of F-4 performance number
F4_upper = 162   # Upper bound of F-4 performance number

# Model Creation
model = pyo.ConcreteModel()

# Variables
model.OF = pyo.Var(bounds=(OF_lower, OF_upper))
model.IR = pyo.Var(bounds=(IR_lower, IR_upper))
model.AAR = pyo.Var(bounds=(AAR_lower, AAR_upper))
model.AY = pyo.Var(bounds=(AY_lower, AY_upper))
model.IM = pyo.Var(bounds=(IM_lower, IM_upper))
model.AS = pyo.Var(bounds=(AS_lower, AS_upper))
model.MON = pyo.Var(bounds=(MON_lower, MON_upper))
model.IOR = pyo.Var(bounds=(IOR_lower, IOR_upper))
model.ADF = pyo.Var(bounds=(ADF_lower, ADF_upper))
model.F4 = pyo.Var(bounds=(F4_lower, F4_upper))

# Objective Function
model.profit = pyo.Objective(
    expr=r * model.AY * model.MON - (c_OF * model.OF + c_IR * model.IR + c_AAR * model.AAR + c_IM * model.IM),
    sense=pyo.maximize
)

# Constraints
model.alkylate_yield = pyo.Constraint(
    expr=(1 - 0.02) * model.AY <= (1 + 0.13167 * model.IOR - 0.00667 * model.IOR**2) * model.OF
)

model.alkylate_yield_l = pyo.Constraint(
    expr=(1 + 0.13167 * model.IOR - 0.00667 * model.IOR**2) * model.OF <= (1 + 0.02) * model.AY
)

model.motor_octane_number = pyo.Constraint(
    expr=(1 - 0.01) * model.MON <= 86.35 + 1.098 * model.IOR - 0.038 * model.IOR**2 + 0.325 * model.AS - 89
)

model.motor_octane_number_l = pyo.Constraint(
    expr=86.35 + 1.098 * model.IOR - 0.038 * model.IOR**2 + 0.325 * model.AS - 89 <= (1 + 0.01) * model.MON
)

model.acid_dilution_factor = pyo.Constraint(
    expr=(1 - 0.05) * model.ADF <= 35.82 - 0.222 * model.F4
)
model.acid_dilution_factor_l = pyo.Constraint(
    expr=35.82 - 0.222 * model.F4 <= (1 + 0.05) * model.ADF
)


model.f4_performance_number = pyo.Constraint(
    expr=(1 - 0.1) * model.F4 <= -133 + 3 * model.MON
)
model.f4_performance_number_l = pyo.Constraint(
    expr=-133 + 3 * model.MON <= (1 + 0.1) * model.F4
)

model.external_isobutane_to_olefin_ratio = pyo.Constraint(
    expr=model.IOR == (model.IR + model.IM) / model.OF
)

model.acid_strength = pyo.Constraint(
    expr=model.AS == (93000 * model.AAR) / (model.AAR * model.ADF + 1000 * model.AAR)
)

model.alkylate_yield_and_feed = pyo.Constraint(
    expr=1.22 * model.AY == model.OF + model.IM
)

# Solve the model
solver = pyo.SolverFactory('ipopt')
results = solver.solve(model)

# Print results
print(results)
print("Profit = ", model.profit())
print("OF = ", model.OF())
print("IR = ", model.IR())
print("AAR = ", model.AAR())
print("AY = ", model.AY())
print("IM = ", model.IM())
print("AS = ", model.AS())
print("MON = ", model.MON())
print("IOR = ", model.IOR())
print("ADF = ", model.ADF())
print("F4 = ", model.F4())

model.name="unknown";
    - termination condition: infeasible
    - message from solver: Ipopt 3.11.1\x3a Converged to a locally infeasible
      point. Problem may be infeasible.

Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 11
  Number of variables: 10
  Sense: unknown
Solver: 
- Status: warning
  Message: Ipopt 3.11.1\x3a Converged to a locally infeasible point. Problem may be infeasible.
  Termination condition: infeasible
  Id: 200
  Error rc: 0
  Time: 0.06941914558410645
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Profit =  0.4020162361351769
OF =  0.006436375158798986
IR =  0.07088369301412772
AAR =  0.0136100669446396
AY =  0.01048293837933738
IM =  0.006352809663992618
AS =  92.82084110386134
MON =  90.0
IOR =  12.0
ADF =  1.9301580766676156
F4 =  152.22222369977234


In [775]:
model.pprint()

10 Var Declarations
    AAR : Size=1, Index=None
        Key  : Lower : Value              : Upper : Fixed : Stale : Domain
        None :     0 : 0.0136100669446396 :   120 : False : False :  Reals
    ADF : Size=1, Index=None
        Key  : Lower : Value              : Upper : Fixed : Stale : Domain
        None :   1.2 : 1.9301580766676156 :     4 : False : False :  Reals
    AS : Size=1, Index=None
        Key  : Lower : Value             : Upper : Fixed : Stale : Domain
        None :    85 : 92.82084110386134 :    93 : False : False :  Reals
    AY : Size=1, Index=None
        Key  : Lower : Value               : Upper : Fixed : Stale : Domain
        None :     0 : 0.01048293837933738 :  5000 : False : False :  Reals
    F4 : Size=1, Index=None
        Key  : Lower : Value              : Upper : Fixed : Stale : Domain
        None :   145 : 152.22222369977234 :   162 : False : False :  Reals
    IM : Size=1, Index=None
        Key  : Lower : Value                : Upper : Fixed 

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [776]:
print(response.text)

**Indices:**

*  None

**Parameters:**

*  Revenue per barrel of alkylate yield:  *r* (dollars/barrel)

**Variables:**

*  Olefin feed (barrels per day):  *OF*
*  Isobutane recycle (barrels per day): *IR*
*  Acid addition rate (thousands of pounds per day): *AAR*
*  Alkylate yield (barrels per day): *AY*
*  Isobutane makeup (barrels per day): *IM*
*  Acid strength (weight percent): *AS*
*  Motor octane number: *MON*
*  External isobutane-to-olefin ratio: *IOR*
*  Acid dilution factor: *ADF*
*  F-4 performance number: *F4*

**Objective Function:**

Maximize daily profit: 
```
Maximize  r * AY * MON - (c_OF * OF + c_IR * IR + c_AAR * AAR + c_IM * IM)
```
where:
* c_OF: Cost per barrel of olefin feed 
* c_IR: Cost per barrel of isobutane recycle
* c_AAR: Cost per thousand pounds of acid addition rate
* c_IM: Cost per barrel of isobutane makeup

**Constraints:**

1. **Alkylate yield:**
   ```
   AY = (1 + 0.13167 * IOR - 0.00667 * IOR^2) * OF
   (1 - 0.02) * AY <= (1 + 0.13167 * IOR - 0.00

In [777]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample Data - Replace with actual values
r = 100  # Revenue per barrel of alkylate yield
c_OF = 50  # Cost per barrel of olefin feed
c_IR = 45  # Cost per barrel of isobutane recycle
c_AAR = 10  # Cost per thousand pounds of acid addition rate
c_IM = 40  # Cost per barrel of isobutane makeup

OF_lower = 1000  # Lower bound of olefin feed
OF_upper = 5000  # Upper bound of olefin feed
IR_lower = 500   # Lower bound of isobutane recycle
IR_upper = 3000  # Upper bound of isobutane recycle
AAR_lower = 10    # Lower bound of acid addition rate
AAR_upper = 100   # Upper bound of acid addition rate
AY_lower = 1200  # Lower bound of alkylate yield
AY_upper = 6000  # Upper bound of alkylate yield
IM_lower = 100   # Lower bound of isobutane makeup
IM_upper = 1000  # Upper bound of isobutane makeup
AS_lower = 85    # Lower bound of acid strength
AS_upper = 95    # Upper bound of acid strength
MON_lower = 88    # Lower bound of motor octane number
MON_upper 